In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import csv
import gmaps

# Google API Key
from config import gkey

# Configure gmaps with API key
gmaps.configure(api_key=gkey)

%matplotlib inline

In [2]:
# Import CSV
g_lookup_pd = pd.read_csv("Resources/Data Filtration.csv")
g_lookup_pd.head()

,restaurant_name,inspection_type,inspection_date,inspection_score,street_number,street_name,street_unit,zip_code,street_address,inspection_month,inspection_year,lat,long,Unique Join
0,2204,Routine,06/28/2019,95,2204,ELM,NaN,75201,3922 SIMPSON ST,Jun 2019,FY2019,39.811585,-94.834959,NaN
1,13209,Routine,08/07/2019,96,13209,C F HAWN,NaN,75253,1910 S HARWOOD ST,Aug 2019,FY2019,32.677500,-96.604348,NaN
2,7-11,Routine,12/18/2019,80,1102,KIEST,NaN,75224,1201 ELM ST #LL09,Dec 2019,FY2020,32.705756,-96.840532,NaN
3,7-11,Routine,04/15/2019,93,225,COLORADO,NaN,75203,4103 LEMMON AVE,Apr 2019,FY2019,32.758575,-96.820548,NaN
4,7-11,Routine,02/25/2019,84,10468,GARLAND,NaN,75218,10969 N STEMMONS FRWY,Feb 2019,FY2019,32.842646,-96.691170,NaN


In [3]:
# Add columns if needed
# Note that we used "" to specify initial entry.

g_lookup_pd["google_lat"] = ""
g_lookup_pd["google_long"] = ""
#g_lookup_pd["col_name"] = ""
#g_lookup_pd["col_name"] = ""
#g_lookup_pd["col_name"] = ""
#g_lookup_pd["col_name"] = ""

g_lookup_pd.head()

,restaurant_name,inspection_type,inspection_date,inspection_score,street_number,street_name,street_unit,zip_code,street_address,inspection_month,inspection_year,lat,long,Unique Join,google_lat,google_long
0,2204,Routine,06/28/2019,95,2204,ELM,NaN,75201,3922 SIMPSON ST,Jun 2019,FY2019,39.811585,-94.834959,NaN,,
1,13209,Routine,08/07/2019,96,13209,C F HAWN,NaN,75253,1910 S HARWOOD ST,Aug 2019,FY2019,32.677500,-96.604348,NaN,,
2,7-11,Routine,12/18/2019,80,1102,KIEST,NaN,75224,1201 ELM ST #LL09,Dec 2019,FY2020,32.705756,-96.840532,NaN,,
3,7-11,Routine,04/15/2019,93,225,COLORADO,NaN,75203,4103 LEMMON AVE,Apr 2019,FY2019,32.758575,-96.820548,NaN,,
4,7-11,Routine,02/25/2019,84,10468,GARLAND,NaN,75218,10969 N STEMMONS FRWY,Feb 2019,FY2019,32.842646,-96.691170,NaN,,


In [4]:
# Rename columns for readability
g_lookup_pd_rn = g_lookup_pd.rename(columns={"restaurant_name": "Restaurant",
                                            "inspection_date": "Inspection Date",
                                            "inspection_score": "Inspection Score",
                                            "street_number": "Street Number",
                                             "street_name": "Street Name",
                                             "zip_code": "Zip",
                                             "street_address": "Street Address",
                                             "lat": "Lat",
                                             "long": "Long",
                                             "google_lat": "G.Lat",
                                             "google_long": "G.Long", 
                                                })
g_lookup_pd_rn

,Restaurant,inspection_type,Inspection Date,Inspection Score,Street Number,Street Name,street_unit,Zip,Street Address,inspection_month,inspection_year,Lat,Long,Unique Join,G.Lat,G.Long
0,2204,Routine,06/28/2019,95,2204,ELM,NaN,75201,3922 SIMPSON ST,Jun 2019,FY2019,39.811585,-94.834959,NaN,,
1,13209,Routine,08/07/2019,96,13209,C F HAWN,NaN,75253,1910 S HARWOOD ST,Aug 2019,FY2019,32.677500,-96.604348,NaN,,
2,7-11,Routine,12/18/2019,80,1102,KIEST,NaN,75224,1201 ELM ST #LL09,Dec 2019,FY2020,32.705756,-96.840532,NaN,,
3,7-11,Routine,04/15/2019,93,225,COLORADO,NaN,75203,4103 LEMMON AVE,Apr 2019,FY2019,32.758575,-96.820548,NaN,,
4,7-11,Routine,02/25/2019,84,10468,GARLAND,NaN,75218,10969 N STEMMONS FRWY,Feb 2019,FY2019,32.842646,-96.691170,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5559,ZOE'S KITCHEN #0031,Routine,11/06/2019,88,2009,ABRAMS,NaN,75214,3312 N BUCKNER BLVD,Nov 2019,FY2020,28.835643,-81.654156,NaN,,
5560,ZOE'S KITCHEN #0031,Routine,05/23/2019,88,2009,ABRAMS,NaN,75214,3312 N BUCKNER BLVD 205,May 2019,FY2019,28.835643,-81.654156,NaN,,
5561,ZRG CONCEPTS,Routine,09/16/2019,96,2505,FITZHUGH,NaN,75204,527 SUNSET AVE,Sep 2019,FY2019,32.812409,-96.783502,NaN,,
5562,ZRG CONCEPTS,Routine,08/22/2019,86,2505,FITZHUGH,NaN,75204,528 S HALL ST,Aug 2019,FY2019,32.812409,-96.783502,NaN,,


In [5]:
# Organize the columns so they are in a more logical order

g_lookup_pd_org = g_lookup_pd_rn[[
    "Restaurant", "Inspection Date", "Inspection Score", "Street Address", "Zip", "Lat", "Long", "G.Lat", "G.Long"]]

g_lookup_pd_org

,Restaurant,Inspection Date,Inspection Score,Street Address,Zip,Lat,Long,G.Lat,G.Long
0,2204,06/28/2019,95,3922 SIMPSON ST,75201,39.811585,-94.834959,,
1,13209,08/07/2019,96,1910 S HARWOOD ST,75253,32.677500,-96.604348,,
2,7-11,12/18/2019,80,1201 ELM ST #LL09,75224,32.705756,-96.840532,,
3,7-11,04/15/2019,93,4103 LEMMON AVE,75203,32.758575,-96.820548,,
4,7-11,02/25/2019,84,10969 N STEMMONS FRWY,75218,32.842646,-96.691170,,
...,...,...,...,...,...,...,...,...,...
5559,ZOE'S KITCHEN #0031,11/06/2019,88,3312 N BUCKNER BLVD,75214,28.835643,-81.654156,,
5560,ZOE'S KITCHEN #0031,05/23/2019,88,3312 N BUCKNER BLVD 205,75214,28.835643,-81.654156,,
5561,ZRG CONCEPTS,09/16/2019,96,527 SUNSET AVE,75204,32.812409,-96.783502,,
5562,ZRG CONCEPTS,08/22/2019,86,528 S HALL ST,75204,32.812409,-96.783502,,


In [6]:
# Apply filters for testing (set limit)

# Create a copy of re-organized Data
g_lookup_pd_filter = g_lookup_pd_org.copy()

# Filter by Inspection Score
g_lookup_pd_filter = g_lookup_pd_filter[g_lookup_pd_filter['Inspection Score'] == 100]

# Filter by Zip
#g_lookup_pd_filter = g_lookup_pd_filter[g_lookup_pd_filter['Zip'] == "75237"]

# Drop null values
#g_lookup_pd_filter = g_lookup_pd_filter.dropna()
#g_lookup_pd_filter = g_lookup_pd_filter.reset_index()

g_lookup_pd_filter

,Restaurant,Inspection Date,Inspection Score,Street Address,Zip,Lat,Long,G.Lat,G.Long
6,"""LET'S DO LUNCH""",08/28/2019,100,2704 ELM ST,75235,32.836816,-96.872095,,
25,"16096 TRINITY BASIN PREPARATORY, INC",10/08/2019,100,5365 SPRING VALLEY RD #101,75237,32.691159,-96.859592,,
28,16159 CORELOGIC MICROMARKET,03/07/2019,100,7995 LBJ STE #118,75063,35.601500,-97.572176,,
47,3131 LOUNGE,09/30/2019,100,8687 N CENTRAL EXPW,75219,40.556674,-75.288257,,
54,50286 SLAYTER'S 50/50 UPSTAIRS BAR (2ND FL),02/15/2019,100,6867 GREENVILLE AVE,75206,32.822958,-96.770158,,
...,...,...,...,...,...,...,...,...,...
5496,WOLFGANG PUCK LEVEL 9 PANTRY,07/08/2019,100,4511 MCKINNEY AVE,75201,32.790322,-96.797076,,
5520,YOGURTLAND,02/05/2019,100,2018 ALLEN ST,75240,32.930161,-96.820963,,
5521,YOGURTLAND - DALLAS (LAKEWOOD),12/11/2019,100,3036 ELM ST #100,75214,28.836494,-81.654504,,
5544,ZAN WESLEY HOLMES JR. MIDDLE,05/24/2019,100,7700 W NORTHWEST HWY,75233,32.712044,-96.866243,,


In [20]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

# Loop through the g_lookup_pd_org and run a lat/long search for each city
for index, row in g_lookup_pd_filter.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/geocode/json"

        street = row['Street Address']
        zip_code = row['Zip']

        # update address key value
        params['address'] = f"{street},{zip_code}"

        # make request
        cities_lat_lng = requests.get(base_url, params=params)

        # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key


        # convert to json
        cities_lat_lng = cities_lat_lng.json()


        
        g_lookup_pd_filter.loc[index, "G.Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        g.lat = g_lookup_pd_filter.loc[index, "G.Lat"]
        
        g_lookup_pd_filter.loc[index, "G.Long"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
        g.long = g_lookup_pd_filter.loc[index, "G.Long"]
        
        if g.lat == "":
            g.lat == "NaN"

    except:
        print("Not found. Keeping calm and carrying on.")

        
        pass
    


Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keepi

Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keepi

Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keeping calm and carrying on.
Not found. Keepi

In [19]:
remove_empty_lat_long = g_lookup_pd_filter.copy()

remove_empty_lat_long.replace.dropna(how='all')

remove_empty_lat_long


,Restaurant,Inspection Date,Inspection Score,Street Address,Zip,Lat,Long,G.Lat,G.Long
6,"""LET'S DO LUNCH""",08/28/2019,100,2704 ELM ST,75235,32.836816,-96.872095,32.7843,-96.7845
25,"16096 TRINITY BASIN PREPARATORY, INC",10/08/2019,100,5365 SPRING VALLEY RD #101,75237,32.691159,-96.859592,32.9399,-96.8145
28,16159 CORELOGIC MICROMARKET,03/07/2019,100,7995 LBJ STE #118,75063,35.601500,-97.572176,32.9254,-96.7701
47,3131 LOUNGE,09/30/2019,100,8687 N CENTRAL EXPW,75219,40.556674,-75.288257,32.8688,-96.7692
54,50286 SLAYTER'S 50/50 UPSTAIRS BAR (2ND FL),02/15/2019,100,6867 GREENVILLE AVE,75206,32.822958,-96.770158,32.871,-96.7659
...,...,...,...,...,...,...,...,...,...
5496,WOLFGANG PUCK LEVEL 9 PANTRY,07/08/2019,100,4511 MCKINNEY AVE,75201,32.790322,-96.797076,32.8216,-96.7885
5520,YOGURTLAND,02/05/2019,100,2018 ALLEN ST,75240,32.930161,-96.820963,32.7955,-96.7953
5521,YOGURTLAND - DALLAS (LAKEWOOD),12/11/2019,100,3036 ELM ST #100,75214,28.836494,-81.654504,32.7853,-96.7794
5544,ZAN WESLEY HOLMES JR. MIDDLE,05/24/2019,100,7700 W NORTHWEST HWY,75233,32.712044,-96.866243,32.8643,-96.7745


In [ ]:
# Organize the columns so they are in a more logical order
drop_old_lat_long = g_lookup_pd_filter.copy()
drop_old_lat_long = drop_old_lat_long[[
    "Restaurant", "Inspection Date", "Inspection Score", "Street Address", "Zip", "G.Lat", "G.Long"]]

# Rename G.Lat and G.Long
drop_old_lat_long_rn = drop_old_lat_long.rename(columns={"G.Lat": "Lat",
                                                        "G.Long": "Long"})
drop_old_lat_long_rn

In [ ]:
# Save as CSV
g_lookup_pd_filter.to_csv("Google Lat Long Error.csv")

In [ ]:
# Plot Heatmap
fig = gmaps.figure()

In [ ]:
# Store 'Lat' and 'Lng' into  locations 
locations = g_lookup_pd_filter[["G.Lat", "G.Lng"]]
inspection_score = g_lookup_pd_filter["Inspection Score"].astype(int)

# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations,weights=inspection_score, 
                                 dissipating=False, max_intensity=80,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig